In [2]:
import psycopg
from sqlalchemy import create_engine
import pandas as pd
import os

credentials = {
    "host": os.getenv("DB_HOST"),
    "dbname": os.getenv("DB_NAME"),
    "port": int(os.getenv("DB_PORT")),
    "user": os.getenv("DB_ADMIN"),
    "password": os.getenv("DB_PW"),
}

# Import data with fantasy scores already tallied
df1 = pd.read_parquet(
    "./Data/Scored NFL Fantasy Rosters for 2024-2025 as of 2024-12-20 1342.parquet",
    engine="pyarrow",
)

In [2]:
# get a list of tables in db with psycopg
with psycopg.connect(**credentials) as conn:
    with conn.cursor() as cur:
        cur.execute(
            """
            SELECT table_name
            FROM information_schema.tables
            WHERE table_schema = 'public'
            """
        )
        table_names = cur.fetchall()
        [print(table[0]) for table in table_names]

nfl_stats
wordle_answers
website_word_list
five_letter_scrabble_words


In [ ]:
# change a table name with psycopg
with psycopg.connect(**credentials) as conn:
    with conn.cursor() as cur:
        cur.execute(
            """
            ALTER TABLE IF EXISTS wordle
            RENAME TO website_word_list
            """
        )

In [3]:
# Create nfl_stats table with psycopg; Unpack credentials with double astericks
with psycopg.connect(**credentials) as conn:
    with conn.cursor() as cur:
        cur.execute(
            """
            CREATE TABLE IF NOT EXISTS nfl_stats (
                roster_index INT,
                position_code TEXT,
                fantasy_owner TEXT,
                fantasy_team TEXT,
                season_type TEXT,
                week INT,
                team_abbr TEXT,
                team_conf TEXT,
                team_division TEXT,
                position_type TEXT,
                position TEXT,
                player_id TEXT,
                player_name TEXT,
                lookup_string TEXT,
                subsequently_traded TEXT,
                stat_label TEXT,
                football_value INT,
                fantasy_points INT
            );
            """
        )

In [7]:
# Query table with psycopg
with psycopg.connect(**credentials) as conn:
    with conn.cursor() as cur:
        cur.execute(
            """
            SELECT *
            FROM nfl_stats as n
            -- WHERE "Word" != 'AALII'
            """
        )
        tmp = cur.fetchall()
        df2 = pd.DataFrame(tmp, columns=[desc[0] for desc in cur.description])
        # df2['Index'] = pd.to_numeric(df2['Index'], errors='coerce').astype('Int64')

In [10]:
# Drop a table with psycopg
with psycopg.connect(**credentials) as conn:
    with conn.cursor() as cur:
        cur.execute(
            """
            DROP TABLE nfl_stats;
            """
        )

In [10]:
# Use Psycopg to Insert into an existing table.
# This approach takes a long time with a lot of transactions so it is not recommended
with psycopg.connect(**credentials) as conn:
    with conn.cursor() as cur:
        for row in range(0,len(df1.iloc[1:10])):  # restrict transactions
            cur.execute(
                """
                INSERT INTO nfl_stats 
                    (roster_index,
                    position_code,
                    fantasy_owner,
                    fantasy_team,
                    season_type,
                    week,
                    team_abbr,
                    team_conf,
                    team_division,
                    position_type,
                    position,
                    player_id,
                    player_name,
                    lookup_string,
                    subsequently_traded,
                    stat_label,
                    football_value,
                    fantasy_points) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);
                """,
                (df1['roster_index'].iloc[row],
                df1['position_code'].iloc[row],
                df1['fantasy_owner'].iloc[row],
                df1['fantasy_team'].iloc[row],
                df1['season_type'].iloc[row],
                df1['week'].iloc[row],
                df1['team_abbr'].iloc[row],
                df1['team_conf'].iloc[row],
                df1['team_division'].iloc[row],
                df1['position_type'].iloc[row],
                df1['position'].iloc[row],
                df1['player_id'].iloc[row],
                df1['player_name'].iloc[row],
                df1['lookup_string'].iloc[row],
                df1['subsequently_traded'].iloc[row],
                df1['stat_label'].iloc[row],
                df1['football_value'].iloc[row],
                df1['fantasy_points'].iloc[row])
            )

In [6]:
# Use SQLAlchemy package to upload to the PostgreSQL
engine = create_engine("postgresql+psycopg://" + credentials['user'] +  ':' + credentials['password'] + '@' + credentials['host'] + ':' + str(credentials['port']) + '/' + credentials['dbname'])

df1.to_sql('nfl_stats', engine, if_exists='replace', index=False)

-1